In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.master("local[*]") \
.appName('test') \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 11:33:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# !for MONTH in {01..12}; wget "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-${MONTH}.csv.gz"

In [3]:
df = spark.read \
      .option("header", 'true')  \
      .csv("fhv_tripdata_2019-10.csv.gz")

In [4]:
df.head(5)

[Row(dispatching_base_num='B00009', pickup_datetime='2019-10-01 00:23:00', dropOff_datetime='2019-10-01 00:35:00', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime='2019-10-01 00:11:29', dropOff_datetime='2019-10-01 00:13:22', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:11:43', dropOff_datetime='2019-10-01 00:37:20', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:56:29', dropOff_datetime='2019-10-01 00:57:47', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:23:09', dropOff_datetime='2019-10-01 00:28:27', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_num

In [6]:
!gzip -dc fhv_tripdata_2019-10.csv.gz > fhv_tripdata_2019-10.csv
!head -n 1001 fhv_tripdata_2019-10.csv > head.csv

In [7]:
!wc -l head.csv

    1001 head.csv


In [8]:
import pandas as pd

In [9]:
df_pandas = pd.read_csv('head.csv', encoding='utf-8')
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [10]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [12]:
from pyspark.sql import types

In [13]:
schema = types.StructType([
  types.StructField('dispatching_base_num', types.StringType(), True),
  types.StructField('pickup_datetime', types.TimestampType(), True),
  types.StructField('dropOff_datetime', types.TimestampType(), True),
  types.StructField('PUlocationID', types.IntegerType(), True),
  types.StructField('DOlocationID', types.IntegerType(), True),
  types.StructField('SR_Flag', types.StringType(), True),
  types.StructField('Affiliated_base_number', types.StringType(), True),
]);

In [14]:
df = spark.read \
.option("header", 'true')  \
.schema(schema) \
.csv("fhv_tripdata_2019-10.csv")

In [15]:
df.head(5)

[Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 23), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 35), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 11, 29), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 13, 22), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00014', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 11, 43), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 37, 20), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 56, 29), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 57, 47), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime=

In [21]:
df.repartition(6)

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropOff_datetime: timestamp, PUlocationID: int, DOlocationID: int, SR_Flag: string, Affiliated_base_number: string]

In [23]:
df.write.parquet('fhv/2019/10')

24/03/04 11:44:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/04 11:44:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/04 11:44:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/04 11:44:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/04 11:44:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [45]:
df = spark.read.parquet('fhv/2019/10')

In [25]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [30]:
df.select('pickup_datetime', 'dropOff_datetime', 'PUlocationID', 'DOlocationID') \
.filter(df.dispatching_base_num == 'B00014') \
.show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|
+-------------------+-------------------+------------+------------+
|2019-10-10 13:01:49|2019-10-10 13:16:15|         264|         264|
|2019-10-10 13:29:34|2019-10-10 13:35:43|         264|         264|
|2019-10-10 13:05:43|2019-10-10 13:57:40|         264|         264|
|2019-10-10 13:00:20|2019-10-10 13:30:05|         264|         264|
|2019-10-10 13:35:34|2019-10-10 14:01:00|         264|         264|
|2019-10-10 13:18:08|2019-10-10 13:50:29|         264|         264|
|2019-10-10 14:55:46|2019-10-10 15:17:05|         264|         264|
|2019-10-10 14:22:44|2019-10-10 14:23:25|         264|         264|
|2019-10-10 14:17:56|2019-10-10 14:47:00|         264|         264|
|2019-10-10 14:03:47|2019-10-10 14:30:46|         264|         264|
|2019-10-10 14:40:33|2019-10-10 15:29:46|         264|         264|
|2019-10-10 14:29:00|2019-10-10 15:15:27|       

In [32]:
import pyspark.sql.functions as F

In [36]:
df.select('pickup_datetime', 'dropOff_datetime', 'PUlocationID', 'DOlocationID') \
.filter(F.date_format(df.pickup_datetime, 'yyyy-MM-dd')  == '2019-10-15') \
.count()

62610

In [54]:
df = df.withColumn('drip_time', (F.col('dropOff_datetime').cast('long') - F.col('pickup_datetime').cast('long')) / 3600)
longest_drip_time = df.agg(F.max('drip_time')).collect()[0][0]
longest_drip_time

631152.5

In [57]:
lookup = spark.read \
      .option("header", 'true')  \
      .csv("taxi_zone_lookup.csv")

In [118]:
joined_df = df.join(lookup, df['PUlocationID'] == lookup['LocationID'], 'inner')

In [119]:
grouped_df = joined_df.groupBy('zone').count()

In [120]:
grouped_df.head(1)

[Row(zone='Homecrest', count=1295)]

In [136]:
grouped_df.filter(grouped_df['count']==grouped_df.agg(F.min('count')).collect()[0][0]).show()

+-----------+-----+
|       zone|count|
+-----------+-----+
|Jamaica Bay|    1|
+-----------+-----+

